#### Setup

In [1]:
from google.colab import drive
import sys

In [2]:
drive.mount('/content/drive')
sys.path.append('/content/drive/Shareddrives/CMPE260/')

Mounted at /content/drive


In [7]:
%run /content/drive/Shareddrives/CMPE260/rec_implementation.ipynb

cpu


#### Training

In [8]:
gaussian_noise = GaussianStrategy()

In [9]:
train_env = Environment(train_matrix)

hits, dcgs = [], []
hits_all, dcgs_all = [], []
step, best_step = 0, 0
step, best_step, best_step_all = 0, 0, 0

users = np.random.permutation(apt_users)

In [10]:
for u in tqdm.tqdm(users):
    print('user =', u)
    user, memory = train_env.reset(u)

    for t in range(int(train_matrix[u].sum())):
      try:
        # given current state, get action using actor network
        action_emb = policy_network(state_representation(user, memory))
        action_emb = gaussian_noise.get_action(action_emb[0], t)
        action = policy_network.get_action(user, torch.tensor(train_env.memory[user.detach().cpu().numpy().astype(int), :]), state_representation, action_emb,
            torch.tensor([item for item in train_env.available_books if item not in train_env.viewed_books]).long()
        )
      except Exception as e:
        print(e)

      # update replay buffer
      user, memory, reward, done = train_env.step(action, action_emb, replay_buffer)

      # update weights of action and critic network
      if len(replay_buffer) > params['batch_size']:
          update(params['batch_size'], params['gamma'], params['min_value'], params['max_value'], params['soft_tau'])

      # monitor training process by evaluating on only one user for efficiency
      if step % 100 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, test_loader)

          hits.append(hit)
          dcgs.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits[best_step], dcgs[best_step]])) >= 0:
              best_step = step // 100
              torch.save(policy_network.state_dict(), params['log_w_gaussian_dir'] + 'policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_gaussian_dir'] + 'value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_gaussian_dir'] + 'state_repr.pth')

      # evaluate for best policy after every 1000 steps on the whole test set
      if step % 1000 == 0 and step > 0:
          hit, dcg = evaluation(policy_network, state_representation, train_env.memory, full_loader)

          hits_all.append(hit)
          dcgs_all.append(dcg)

          if np.mean(np.array([hit, dcg]) - np.array([hits_all[best_step_all], dcgs_all[best_step_all]])) >= 0:
              best_step_all = step // 1000
              torch.save(policy_network.state_dict(), params['log_w_gaussian_dir'] + 'best_policy_net.pth')
              torch.save(value_network.state_dict(), params['log_w_gaussian_dir'] + 'best_value_net.pth')
              torch.save(state_representation.state_dict(), params['log_w_gaussian_dir'] + 'best_state_repr.pth')

      step += 1

  0%|          | 0/52 [00:00<?, ?it/s]

user = 162


  2%|▏         | 1/52 [00:12<10:32, 12.41s/it]

user = 226


  4%|▍         | 2/52 [00:24<10:11, 12.23s/it]

...

 96%|█████████▌| 50/52 [39:06<00:47, 23.61s/it]

user = 27


 98%|█████████▊| 51/52 [39:13<00:18, 18.74s/it]

user = 132


100%|██████████| 52/52 [43:48<00:00, 50.54s/it]


In [11]:
torch.save(policy_network.state_dict(), params['log_w_gaussian_dir'] + 'policy_net_final.pth')
torch.save(value_network.state_dict(), params['log_w_gaussian_dir'] + 'value_net_final.pth')
torch.save(state_representation.state_dict(), params['log_w_gaussian_dir'] + 'state_repr_final.pth')

In [12]:
with open(params['log_w_gaussian_dir'] + 'memory.pickle', 'wb') as f:
  pickle.dump(train_env.memory, f)

#### Evaluation on test set

In [13]:
with open(params['log_w_gaussian_dir'] + 'memory.pickle', 'rb') as f:
    memory = pickle.load(f)

In [14]:
state_rep = State_Representation(num_of_users, num_of_books, params['embedding_dim'], params['hidden_dim'])
policy = Actor(params['embedding_dim'], params['hidden_dim'])
state_rep.load_state_dict(torch.load(params['log_w_gaussian_dir'] + 'best_state_repr.pth'))
policy.load_state_dict(torch.load(params['log_w_gaussian_dir'] + 'best_policy_net.pth'))

<All keys matched successfully>

In [15]:
hit, dcg = evaluation(policy, state_rep, memory, full_loader)
print('hit rate: ', hit, 'dcg: ', dcg)

hit rate:  0.9741984015044663 dcg:  0.48333943031368515


#### Inference

In [ ]:
random_user = test_data['user'].sample().iloc[0]
random_user

89

In [ ]:
temp_test_data = test_data[test_data.user == random_user]
temp = pd.merge(temp_test_data, filtered_df, on=['user', 'book'])
temp

,user,book,isbn,title,author,user_id,rating,best_seller
0,89,154,385416342,The Firm,John Grisham,101851,8.0,True
1,89,679,399501487,Lord of the Flies,William Gerald Golding,101851,8.0,True
2,89,845,441328008,"Heretics of Dune (Dune Chronicles, Book 5)",Frank Herbert,101851,7.0,True
3,89,846,441294677,"God Emperor of Dune (Dune Chronicles, Book 4)",Frank Herbert,101851,7.0,False
4,89,1030,618002219,The Hobbit: or There and Back Again,J.R.R. Tolkien,101851,10.0,True
5,89,1283,811802981,The Golden Mean: In Which the Extraordinary Co...,Nick Bantock,101851,10.0,True
6,89,1873,439139597,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,101851,10.0,True
7,89,3115,60652934,The Screwtape Letters,C. S. Lewis,101851,9.0,True
8,89,3451,761539220,The Sims Vacation: Expansion Pack: Prima's Off...,David Chong,101851,9.0,True
9,89,3916,345338588,"On a Pale Horse (Incarnations of Immortality, ...",Piers Anthony,101851,9.0,False


In [ ]:
recommendations = []

for model, state_representation in zip([policy], [state_rep]):
    env = Environment(test_matrix)
    user, memory = env.reset(random_user)

    # give 2 books that the user has rated in the past
    user, memory, reward, _ = env.step(torch.tensor([27873]))
    user, memory, reward, _ = env.step(torch.tensor([27879]))

    # get next 3 recommendations based on the two books above
    recs = []
    for i in range(3):
        action_emb = model(state_representation(user, memory))
        action = model.get_action(
            user,
            torch.tensor(env.memory[user.detach().cpu().numpy().astype(int), :]),
            state_representation,
            action_emb,
            torch.tensor([item for item in env.available_books if item not in env.viewed_books]).long()
        )
        user, memory, reward, _ = env.step(action)
        recs.append(action)

    recommendations.append(recs)

In [ ]:
recos = [r.item() for r in recommendations[0]]
recos

[1873, 10121, 17830]

In [ ]:
recommended_books = filtered_df[filtered_df.book.isin(recos)][['title', 'author', 'best_seller']].drop_duplicates()
recommended_books

,title,author,best_seller
67764,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,True
185486,Dead Man's Mirror,Agatha Christie,True
257056,Village Square (Homespun),Linda Eberhardt,True
